### Setup

In [1]:
import numpy as np

from hot_dust import preprocess, model

In [2]:
from importlib import reload

preprocess = reload(preprocess)
model = reload(model)

In [3]:
ds = preprocess.prepare_training_data()
train, validate, test = preprocess.to_tensorflow(preprocess.split_training_data(ds))

### Parameters

In [4]:
epochs = 150 # Max 500
batch_size = 64
buffer_size = 10 * batch_size

model = reload(model)  # DEBUG

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size)

layer = model.pretraining(dataset=test)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [5]:
network = model.compile(normalization=layer)
network.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 normalization (Normalizatio  (None, 9)                19        
 n)                                                              
                                                                 
 dense (Dense)               (None, 8)                 80        
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 180
Trainable params: 161
Non-trainable params: 19
______________________________________________________________

### Fitting

In [6]:
import tensorflow as tf 
early_stopping = tf.keras.callbacks.EarlyStopping( 
    monitor='val_loss',
    #min_delta=0,
    patience=20,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0
)
# List for callbacks
callbacks_list = [early_stopping]

fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2, 
    callbacks = callbacks_list
)

network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", fit)

Epoch 1/150
3122/3122 - 7s - loss: 0.7311 - val_loss: 0.4334 - 7s/epoch - 2ms/step
Epoch 2/150
3122/3122 - 8s - loss: 0.3813 - val_loss: 0.3469 - 8s/epoch - 3ms/step
Epoch 3/150
3122/3122 - 7s - loss: 0.3233 - val_loss: 0.3010 - 7s/epoch - 2ms/step
Epoch 4/150
3122/3122 - 4s - loss: 0.2850 - val_loss: 0.2688 - 4s/epoch - 1ms/step
Epoch 5/150
3122/3122 - 6s - loss: 0.2562 - val_loss: 0.2426 - 6s/epoch - 2ms/step
Epoch 6/150
3122/3122 - 6s - loss: 0.2333 - val_loss: 0.2226 - 6s/epoch - 2ms/step
Epoch 7/150
3122/3122 - 7s - loss: 0.2167 - val_loss: 0.2090 - 7s/epoch - 2ms/step
Epoch 8/150
3122/3122 - 7s - loss: 0.2061 - val_loss: 0.2005 - 7s/epoch - 2ms/step
Epoch 9/150
3122/3122 - 7s - loss: 0.1990 - val_loss: 0.1956 - 7s/epoch - 2ms/step
Epoch 10/150
3122/3122 - 5s - loss: 0.1943 - val_loss: 0.1912 - 5s/epoch - 2ms/step
Epoch 11/150
3122/3122 - 5s - loss: 0.1908 - val_loss: 0.1883 - 5s/epoch - 2ms/step
Epoch 12/150
3122/3122 - 5s - loss: 0.1883 - val_loss: 0.1858 - 5s/epoch - 1ms/step
E

INFO:tensorflow:Assets written to: data/network\assets


INFO:tensorflow:Assets written to: data/network\assets


### Training and Validation Losses

In [7]:
model.plot_loss(fit)

:NdOverlay   [Variable]
   :Scatter   [epoch]   (value)